In [1]:
import pandas as pd
import networkx as nx
import plotly.express as px


In [8]:
df_nodes = pd.read_csv('nodes.csv')
df_edges = pd.read_csv('links.csv')

In [10]:


def create_dynamic_graphs(df_nodes, df_edges, periods):
    graphs = {}
    for period in periods:
        G = nx.DiGraph()
        
        # Adicionando nós, filtrando por founding_date
        for _, row in df_nodes[df_nodes['founding_date'] <= period].iterrows():
            G.add_node(row['id'], **row.to_dict())
        
        # Adicionando arestas, filtrando por start_date e end_date
        for _, row in df_edges[(df_edges['start_date'] <= period) & 
                               ((df_edges['end_date'].isna()) | (df_edges['end_date'] >= period))].iterrows():
            G.add_edge(row['source'], row['target'], **row.to_dict())
        
        graphs[period] = G
    return graphs

# Definindo os períodos de interesse
periods = pd.date_range(start='2020-01-01', end='2023-12-31', freq='Q').strftime('%Y-%m').tolist()

# Criando grafos para cada período
graphs = create_dynamic_graphs(df_nodes, df_edges, periods)


C:\Users\maico\AppData\Local\Temp\ipykernel_3604\1294282240.py:21: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  periods = pd.date_range(start='2020-01-01', end='2023-12-31', freq='Q').strftime('%Y-%m').tolist()


In [11]:
centrality_data = []

for period, G in graphs.items():
    eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    
    for node in G.nodes():
        centrality_data.append({
            'period': period,
            'node': node,
            'eigenvector': eigenvector_centrality.get(node, 0),
            'degree': degree_centrality.get(node, 0),
            'betweenness': betweenness_centrality.get(node, 0)
        })

centrality_df = pd.DataFrame(centrality_data)


In [13]:
#centrality_df.to_csv('centrality_new.csv', index=False)
centrality_df = pd.read_csv('centrality_new.csv')
centrality_df['period'] = pd.to_datetime(centrality_df['period'])
centrality_df.head()

,period,node,eigenvector,degree,betweenness
0,2020-03-01,"Abbott, Mcbride and Edwards",0.000024,0.000105,0.0
1,2020-03-01,Abbott-Gomez,0.000008,0.000035,0.0
2,2020-03-01,Abbott-Ibarra,0.000016,0.000070,0.0
3,2020-03-01,Abbott-Sullivan,0.000032,0.000141,0.0
4,2020-03-01,Acevedo PLC,0.000048,0.000211,0.0


In [14]:
# Filtrar para incluir apenas dados anuais (primeiro dia do ano)
#centrality_df = centrality_df[centrality_df['period'].dt.month == 1]

# Garantir que a coluna 'period' esteja no formato de string para plotly
#centrality_df['period'] = centrality_df['period'].dt.to_period('Y').astype(str)

In [ ]:
#node_seafood_df = centrality_df[centrality_df['node'] == 'SouthSeafood Express Corp']

In [21]:
# Opcional: Filtrar para incluir apenas os nós mais importantes
important_nodes = centrality_df.groupby('node')['eigenvector'].mean().sort_values(ascending=False).head(10).index
centrality_df_filtered = centrality_df[centrality_df['node'].isin(important_nodes)]


In [22]:
import plotly.express as px

# Visualização para Eigenvector Centrality
fig = px.line(centrality_df_filtered, x='period', y='eigenvector', color='node', title='Eigenvector Centrality Over Time')
fig.show()

# Visualização para Degree Centrality
fig = px.line(centrality_df_filtered, x='period', y='degree', color='node', title='Degree Centrality Over Time')
fig.show()

# Visualização para Betweenness Centrality
fig = px.line(centrality_df_filtered, x='period', y='betweenness', color='node', title='Betweenness Centrality Over Time')
fig.show()


In [ ]:
df_centrality_old = pd.read_csv('centrality.csv')
df_centrality_old

In [43]:
#rename Unnamed: 0 to company
df_centrality_old = df_centrality_old.rename(columns={'Unnamed: 0':'Node'})
df_centrality_old = df_centrality_old.rename(columns={'Betweenness Centrality_before':'Betweenness_Centrality_before'})
df_centrality_old = df_centrality_old.rename(columns={'Betweenness Centrality_after':'Betweenness_Centrality_after'})


In [46]:
#company com nome southseafood
df_centrality_old[df_centrality_old['Node'] == 'SouthSeafood Express Corp']

,Node,Degree_Centrality_before,Eigenvector_Centrality_before,Betweenness_Centrality_before,Degree_Centrality_after,Eigenvector_Centrality_after,Betweenness_Centrality_after
60463,SouthSeafood Express Corp,0.000017,8.506850e-98,0.0,0.000017,5.173217e-11,0.0


In [50]:
# Calcular a média das centralidades para cada nó
df_centrality_old['Degree_Centrality_mean'] = df_centrality_old[['Degree_Centrality_before', 'Degree_Centrality_after']].mean(axis=1)
df_centrality_old['Eigenvector_Centrality_mean'] = df_centrality_old[['Eigenvector_Centrality_before', 'Eigenvector_Centrality_after']].mean(axis=1)
df_centrality_old['Betweenness_Centrality_mean'] = df_centrality_old[['Betweenness_Centrality_before', 'Betweenness_Centrality_after']].mean(axis=1)

# Selecionar os top 10 nós baseados na média das centralidades
top_10_degree = df_centrality_old.sort_values(by='Degree_Centrality_mean', ascending=False).head(10)
top_10_eigenvector = df_centrality_old.sort_values(by='Eigenvector_Centrality_mean', ascending=False).head(10)
top_10_betweenness = df_centrality_old.sort_values(by='Betweenness_Centrality_mean', ascending=False).head(10)

# Combinar os top 10 nós em um DataFrame único, removendo duplicatas
top_nodes = pd.concat([top_10_degree, top_10_eigenvector, top_10_betweenness]).drop_duplicates().reset_index(drop=True)

# Exibir os top nós selecionados
print(top_nodes)


                           Node  Degree_Centrality_before  \
0                   Downs Group                  0.001983   
1                  Sandra Young                  0.001504   
2               Anne Villanueva                  0.001504   
3              Jennifer Gardner                  0.001504   
4                    Anna Davis                  0.001504   
5    Mills, Atkinson and Chavez                  0.001520   
6                 Melissa Davis                  0.001487   
7                 Joseph Gentry                  0.001487   
8                 Breanna Price                  0.001487   
9              Kenneth Gonzales                  0.001487   
10         Polák Procházka a.s.                  0.000000   
11             Catherine Macias                  0.001487   
12                   Erica Ford                  0.001487   
13                Kelsey Ortega                  0.001487   
14                  Sharon Moon                  0.000463   
15             Steven Ro

In [51]:
import plotly.graph_objects as go

# Função para criar gráficos comparativos de barras para os top nós
def plot_comparison(df, metric):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=df['Node'],
        y=df[f'{metric}_before'],
        name=f'{metric.replace("_", " ")} Before'
    ))
    fig.add_trace(go.Bar(
        x=df['Node'],
        y=df[f'{metric}_after'],
        name=f'{metric.replace("_", " ")} After'
    ))
    
    fig.update_layout(
        title=f'Comparison of {metric.replace("_", " ")} Before and After for Top 10 Nodes',
        xaxis_title='Node',
        yaxis_title=metric.replace("_", " "),
        barmode='group'
    )
    
    fig.show()

# Filtrar o DataFrame para apenas os top nós
def filter_top_nodes(df, top_nodes):
    return df[df['Node'].isin(top_nodes['Node'])]

# Filtrar os dados
top_degree_df = filter_top_nodes(df_centrality_old, top_10_degree)
top_eigenvector_df = filter_top_nodes(df_centrality_old, top_10_eigenvector)
top_betweenness_df = filter_top_nodes(df_centrality_old, top_10_betweenness)

# Plotar as comparações
plot_comparison(top_degree_df, 'Degree_Centrality')
plot_comparison(top_eigenvector_df, 'Eigenvector_Centrality')
plot_comparison(top_betweenness_df, 'Betweenness_Centrality')
